In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import sys
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
sys.path.append(parent_dir)

from google import genai
from scripts.embedding_helpers import compute_similarity

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [4]:
from scripts.helpers import get_only_generated_info, load_courses

# courses, ctoi = load_courses("../data/formatted/fi.json")
# courses2, ctoi2 = load_courses("../data/formatted/přf.json")

# courses = courses + courses2

# for key, value in ctoi2.items():
#     ctoi2[key] = value + len(ctoi)

# ctoi.update(ctoi2)

courses, ctoi = load_courses("data/formatted")

print(len(courses))
# courses_generated_info = get_only_generated_info(courses)

21106


In [5]:
print(courses[100])

{'CODE': ' LSP_TA_S ', 'FACULTY': ' CST ', 'NAME': ' LAP and LSP teacher assistant - Spanish ', 'LANGUAGE': ' španělština ', 'SEMESTER': ' podzim 2024 ', 'CREDITS': ' 2 ', 'DEPARTMENT': ' ', 'TEACHERS': ' Alchazidu, A. - De Azevedo Camacho, V. - Žváčková, J. - Holasová, M. ', 'COMPLETION': ' z ', 'PREREQUISITES': ' ', 'FIELDS_OF_STUDY': ' ', 'TYPE_OF_STUDY': ' ', 'LECTURES_SEMINARS_HOMEWORK': ' 0/2/0 ', 'SYLLABUS': ' ', 'OBJECTIVES': ' The course is intended for students who are native speakers of Spanish language and seek experience in language teaching. ', 'TEXT_PREREQUISITS': ' ', 'ASSESMENT_METHODS': ' ', 'TEACHING_METHODS': ' Students will be expected to cooperate with a Language Centre teacher on preparation as well as delivery of language classes for academic or specific purposes. Each of the students will be contacted by one Language Centre teacher. The teacher will be in charge of the course and responsible for its quality. S/he will guide the student in terms of language teac

In [18]:
from scripts.helpers import edit_catalogue_for_llm, dict_print
from scripts.embedding_helpers import embed
import matplotlib.pyplot as plt
import numpy as np

sample_courses_names = ["IV109", "IV111", "MB152", "VB035", "PV021", "VV024", "PV168", "PB130", "PB152", "PB152zk", "PV066", "IB031"]
sample_courses_ids = [ctoi[name] for name in sample_courses_names]
samples_courses = [courses[i] for i in sample_courses_ids]

for course in samples_courses:
    print(dict_print(edit_catalogue_for_llm(course)))

# sample_embeds = [embed(client.models, dict_print(edit_catalogue_for_llm(course))).embeddings[0].values for course in samples_courses]

# # Plot similarities

# similarity_matrix = np.zeros((len(samples_courses), len(samples_courses)))
# for i in range(len(samples_courses)):
#     for j in range(len(samples_courses)):
#         similarity_matrix[i, j] = compute_similarity(sample_embeds[i], sample_embeds[j])

# plt.imshow(similarity_matrix, interpolation='nearest', cmap='coolwarm')
# plt.xticks(range(len(samples_courses)), sample_courses_names, rotation=45)
# plt.yticks(range(len(samples_courses)), sample_courses_names)
# plt.colorbar()
# plt.show()

NAME: Modeling and Simulation
SYLLABUS: Introduction, history, role of modeling and simulation in research, applications. Computational models.     Complex systems, system thinking, feedback loops.     System dynamics approach, examples (demographics, Limits to growth).     Agent based modeling: basic principles, cellular automata, decentralized systems.     Game theory, models of cooperation. Models of adaptation (genetic algorithms, neural networks).     Modeling of networks: examples of networks and their properties, models of networks.     Analysis and evaluation of models.     Application of modeling from different areas (e.g. economics, traffic, epidemiology, biology).
OBJECTIVES: The course offers a wide overview of computational modeling and gives students a practical experience with computational modeling.
LEARNING_OUTCOMES: At the end of the course students will be able to: describe main concepts of complex systems (particularly "feedback loops"); explain main principles and 

In [7]:
all_embeds = []
print(len(courses))

21106


In [8]:
# import time
# from scripts.helpers import edit_catalogue_for_llm, dict_print

# for i, course in enumerate(courses):
#     if i < 2202:
#         continue
#     if i % 10 == 0:
#         print(i)
#     time.sleep(0.15)
#     all_embeds.append(embed(client.models, dict_print(edit_catalogue_for_llm(course))).embeddings[0].values)
#     # if i % 100 == 0 and i != 0:
#     #     np.save(f"../data/embeddings/embeds{i}.npy", all_embeds)

# np.save(f"../data/embeddings/ff.npy", all_embeds)

In [9]:
# print(len(all_embeds))
# np.save(f"../data/embeddings/ff_.npy", all_embeds)

In [11]:
# fi = np.load(f"../data/embeddings/fi.npy", allow_pickle=True)
# print(len(fi))
# prf = np.load(f"../data/embeddings/přf.npy", allow_pickle=True)
# print(len(prf))
# all_embeds = np.concatenate((fi, prf), axis=0)

all_embeds = np.load(f"data/embeddings/embeds_from_catalogue_reduced.npy", allow_pickle=True)

In [12]:
from scripts.embedding_helpers import courses_codes_to_indices, recommend_based_on_liked_disliked

course_names_neg = []
# course_names_pos = ["PV021", "PV197"]
# course_names_neg = ["IB111"]
course_names_pos = ["MA010", "IA159", "PV021", "PB156", "IB031", "PB016", "IV109", "IB107", "PB161"]
course_names_neg = ["PB007", "PV080"]
# course_names_pos = ["PV168"]
# course_names_neg = ["IB111"]

# course_names_pos = ["PV005", "PV198", "PV157", "PB176", "PJ_03", "PJ_04", "p947", "IB109", "PV197", "IB031"]
# course_names_neg = ["MV008"]
# course_names_pos = ["PB071", "MB152", "PV021", "NJII_641", "IB002"]
# course_names_neg = ["PB007", "PV080", "MB152", "IB000", "PB152cv"]

# normalize all_embeds - subtract mean and divide by std
all_embeds_normalized = np.array(all_embeds)
all_embeds_normalized = (all_embeds - all_embeds.mean(axis=0)) / all_embeds.std(axis=0)

# Find embeds closes to 0
# smallest_embeds = np.argsort(np.linalg.norm(all_embeds, axis=1))
# for i in smallest_embeds[:10]:
#     print(courses[i]["NAME"], courses[i]["CODE"])

# find most similar courses
recommended_course_indices_old, simlarities_old = recommend_based_on_liked_disliked(course_names_pos, course_names_neg, all_embeds_normalized, ctoi, 100, "old")
recommended_course_indices_new, simlarities_new = recommend_based_on_liked_disliked(course_names_pos, course_names_neg, all_embeds_normalized, ctoi, 100, "new")

# Remove courses from positive and negative lists
recommended_course_indices_old = [i for i in recommended_course_indices_old if courses[i]["CODE"].strip() not in course_names_pos and courses[i]["CODE"].strip() not in course_names_neg]
recommended_course_indices_new = [i for i in recommended_course_indices_new if courses[i]["CODE"].strip() not in course_names_pos and courses[i]["CODE"].strip() not in course_names_neg]

for old_index, new_index in zip(recommended_course_indices_old[:20], recommended_course_indices_new[:20]):
    print(f"{courses[old_index]['NAME']:<70} {courses[old_index]['CODE']:<10} {courses[new_index]['NAME']:<70} {courses[new_index]['CODE']}")

                                                                        PHV444en   Complexity                                                             IA012 
 Mathematical Foundations of Computer Science                           IB000                                                                             PHV444en 
 Mathematical Foundations of Computer Science                           IB000ext   Introduction to Informatics                                            IB110 
 Complexity                                                             IA012      Mathematical Foundations of Computer Science                           IB000 
 Introduction to Informatics                                            IB110      Mathematical Foundations of Computer Science                           IB000ext 
 Formal Languages and Automata                                          IB005      Graph Theory                                                           M5140 
 Algorithmic game theory    

In [13]:
# faculties = []
# for course in courses:
#     if course["FACULTY"] not in faculties:
#         faculties.append(course["FACULTY"])

# print(faculties)

# all_embdeds = []
# for fac in faculties:
#     fac_embeds = np.load(f"../data/embeddings/{fac.lower().strip()}.npy", allow_pickle=True)
#     all_embdeds.extend(fac_embeds)

# print(len(all_embdeds))

# all_embdeds = np.array(all_embdeds, dtype=np.float32)
# np.save(f"../data/embeddings/embeds_from_catalogue_reduced.npy", all_embdeds)

In [14]:
# # Cluster all_embeds

# from sklearn.cluster import KMeans

# for n in range(5, 20):
#     kmeans = KMeans(n_clusters=n, random_state=0).fit(all_embeds_normalized)
#     print(n, kmeans.inertia_)

In [15]:
# kmeans = KMeans(n_clusters=50, random_state=0).fit(all_embeds_normalized)

# for i in range(kmeans.n_clusters):
#     # Get the course at the center of the cluster
#     center = kmeans.cluster_centers_[i]
#     closest = np.argmin(np.linalg.norm(all_embeds_normalized - center, axis=1))
#     print(f"Cluster {i}: {courses[closest]['NAME']}")

In [16]:
# from sklearn.decomposition import PCA

# for i in range(50, 700, 20):
#     pca = PCA(n_components=i)
#     pca.fit(all_embeds_normalized)
#     print(i, pca.explained_variance_ratio_.sum())

In [17]:
# pca = PCA(n_components=370)
# pca.fit(all_embeds_normalized)

# # Save the transformed data
# all_embeds_pca = pca.transform(all_embeds_normalized)

# np.save(f"../data/embeddings/embeds_from_catalogue_reduced_pca_370.npy", all_embeds_pca)